In [1]:
import os, sys
sys.path.append(os.path.dirname(os.getcwd()))
from utils import *
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
df = pd.read_csv("../data/cl_papers.csv", index_col=0)
df

,id,title,area,source,year,doi,abstract
0,main.1004,AnswerFact: Fact Checking in Product Question ...,Question Answering,EMNLP,2020,10.18653/v1/2020.emnlp-main.1004,Product-related question answering platforms n...
1,main.1006,Knowledge-Grounded Dialogue Generation with Pr...,Dialog and Interactive Systems,EMNLP,2020,10.18653/v1/2020.emnlp-main.1006,We study knowledge-grounded dialogue generatio...
2,main.1009,BiST: Bi-directional Spatio-Temporal Reasoning...,Dialog and Interactive Systems,EMNLP,2020,10.18653/v1/2020.emnlp-main.1009,Video-grounded dialogues are very challenging ...
3,main.1010,A Knowledge-Aware Sequence-to-Tree Network for...,NLP Applications,EMNLP,2020,10.18653/v1/2020.emnlp-main.1010,With the advancements in natural language proc...
4,main.1011,Knowledge Association with Hyperbolic Knowledg...,Information Extraction,EMNLP,2020,10.18653/v1/2020.emnlp-main.1011,Capturing associations for knowledge graphs (K...
...,...,...,...,...,...,...,...
9250,T4773,Rank-Aware Negative Training for Semi-Supervis...,Machine Learning for NLP,ACL,2023,NaN,NaN
9251,T4777,Transparency Helps Reveal When Language Models...,"Linguistic Theories, Cognitive Modeling, and P...",ACL,2023,NaN,NaN
9252,T4803,Design Choices for Crowdsourcing Implicit Disc...,Discourse and Pragmatics,ACL,2023,NaN,NaN
9253,T4929,Time-and-Space-Efficient Weighted Deduction,"Semantics: Sentence-level Semantics, Textual I...",ACL,2023,NaN,NaN


In [38]:
from tqdm import tqdm
tqdm.pandas()


df.loc[df['doi'].isnull(), 'doi'] = df['title'][df['doi'].isnull()].progress_apply(get_acl_anthology_doi)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1118/1118 [00:17<00:00, 64.52it/s]


In [39]:
df[pd.isna(df['doi'])]

,id,title,area,source,year,doi,abstract,semantic_scholar_id
77,main.1383,Identifying Elements Essential for BERT's Mult...,Machine Translation and Multilinguality,EMNLP,2020,None,It has been shown that multilingual BERT (mBER...,c3a662b864673d8cc7469051419ab8819926d4b0
154,main.1670,Does my multimodal model learn cross-modal int...,"Language Grounding to Vision, Robotics and Beyond",EMNLP,2020,None,Modeling expressive cross-modal interactions s...,9c16dcbcdfe6991f5d448543e6f4cbdf37149883
188,main.1837,Don't Read Too Much Into It: Adaptive Computat...,Question Answering,EMNLP,2020,None,Most approaches to Open-Domain Question Answer...,0c005cf844ca0618eb4ae304e8fa071c405e54f6
191,main.1862,Let's Stop Incorrect Comparisons in End-to-end...,Information Extraction,EMNLP,2020,None,Despite efforts to distinguish three different...,None
203,main.1935,Don't Neglect the Obvious: On the Role of Unam...,Semantics: Lexical Semantics,EMNLP,2020,None,State-of-the-art methods for Word Sense Disamb...,None
...,...,...,...,...,...,...,...,...
9171,P823,Where's the Point? Self-Supervised Multilingua...,Multilingualism and Cross-Lingual NLP,ACL,2023,None,Many NLP pipelines split text into sentences a...,NaN
9174,P834,"On ""Scientific Debt"" in NLP: A Case for More R...",Theme: Reality Check,ACL,2023,None,This evidence-based position paper critiques c...,066a9da13badca3791832f50f47103f31d681189
9179,P855,CFSum: Coarse-to-Fine Contribution Network for...,Summarization,ACL,2023,None,Multimodal summarization usually suffers from ...,NaN
9198,P942,"On Second Thought, Let's Not Think Step by Ste...",Ethics and NLP,ACL,2023,None,Generating a Chain of Thought (CoT) has been s...,NaN


In [43]:
valid_dois = df[df['doi'].notnull()]['doi'].tolist()
paper_details = bulk_get_paper_details(valid_dois)


Fetching papers: 18it [00:26,  1.47s/it]


In [46]:
doi_to_paper_id = {doi: detail.paper_id for doi, detail in zip(valid_dois, paper_details) if detail is not None}
df['semantic_scholar_id'] = df['doi'].map(doi_to_paper_id).combine_first(df['semantic_scholar_id'])

In [47]:
df[pd.isna(df['semantic_scholar_id'])]

,id,title,area,source,year,doi,abstract,semantic_scholar_id
77,main.1383,Identifying Elements Essential for BERT's Mult...,Machine Translation and Multilinguality,EMNLP,2020,None,It has been shown that multilingual BERT (mBER...,NaN
152,main.1658,Generating similes effortlessly like a Pro: A ...,Language Generation,EMNLP,2020,10.18653/v1/2020.emnlp-main.524,"Literary tropes, from poetry to stories, are a...",NaN
154,main.1670,Does my multimodal model learn cross-modal int...,"Language Grounding to Vision, Robotics and Beyond",EMNLP,2020,None,Modeling expressive cross-modal interactions s...,NaN
188,main.1837,Don't Read Too Much Into It: Adaptive Computat...,Question Answering,EMNLP,2020,None,Most approaches to Open-Domain Question Answer...,NaN
191,main.1862,Let's Stop Incorrect Comparisons in End-to-end...,Information Extraction,EMNLP,2020,None,Despite efforts to distinguish three different...,NaN
...,...,...,...,...,...,...,...,...
9206,P976,LiveChat: A Large-Scale Personalized Dialogue ...,Resources and Evaluation,ACL,2023,10.18653/v1/2023.acl-long.858,Open-domain dialogue systems have made promisi...,NaN
9207,P987,ThinkSum: Probabilistic reasoning over sets us...,Large Language Models,ACL,2023,10.18653/v1/2023.acl-long.68,Large language models (LLMs) have a substantia...,NaN
9208,P989,Unsupervised Discontinuous Constituency Parsin...,"Syntax: Tagging, Chunking, and Parsing",ACL,2023,10.18653/v1/2023.acl-long.316,We study grammar induction with mildly context...,NaN
9210,P998,Songs Across Borders: Singable and Controllabl...,NLP Applications,ACL,2023,10.18653/v1/2023.acl-long.27,The development of general-domain neural machi...,NaN


In [48]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


def search_paper_by_title(title):
    base_url = 'https://api.semanticscholar.org/graph/v1/paper/search'
    response = requests.get(base_url, params={'query': title}, headers={"x-api-key": API_KEY})
    
    paper_dict = response.json()
    if 'data' not in paper_dict:
        return None
        
    for result in paper_dict['data']:
        if similar(title, result['title']) > 0.85:
            return result['paperId']
    


search_paper_by_title('Document Modeling with External Attention for Sentence Extraction')

'98346b7de76e905d9f23064cf02606e4771e03c4'

In [74]:
def update_row(row):
    if pd.isna(row['semantic_scholar_id']):
        paper_id = search_paper_by_title(row['title'].lower().replace('{','').replace('}',''))
        if paper_id:
            row['semantic_scholar_id'] = paper_id
    return row

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    if pd.isna(row['semantic_scholar_id']):
        updated_row = update_row(row)
        df.loc[index] = updated_row

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9255/9255 [01:01<00:00, 149.50it/s]


In [89]:
df[pd.isna(df['semantic_scholar_id'])]

,id,title,area,source,year,doi,abstract,semantic_scholar_id
4073,257,An Information-theoretic Approach to Prompt En...,Machine Learning for NLP,ACL,2022,10.18653/v1/2022.acl-long.60,NaN,NaN
4772,130,Baize: An Open-Source Chat Model with Paramete...,Dialogue and Interactive Systems,EMNLP,2023,10.18653/v1/2023.emnlp-main.385,NaN,NaN
4978,1298,MULTITuDE: Large-Scale Multilingual Machine-Ge...,Natural Language Generation,EMNLP,2023,10.18653/v1/2023.emnlp-main.616,NaN,NaN
6157,1542,Understanding Back-Translation at Scale,Area G (Machine Translation and Multilingualit...,EMNLP,2018,10.18653/v1/D18-1045,NaN,NaN
7134,1334,Social IQa: Commonsense Reasoning about Social...,Reasoning,EMNLP,2019,10.18653/v1/D19-1454,NaN,NaN
7492,712,When a Good Translation is Wrong in Context: C...,Machine Translation,ACL,2019,10.18653/v1/P19-1116,NaN,NaN


In [76]:
for title in list(df[pd.isna(df['semantic_scholar_id'])]['title']):
    print(f'"{title}": None,')

"Let's Stop Incorrect Comparisons in End-to-end Relation Extraction!": None,
"DAGA: Data Augmentation with a Generation Approach forLow-resource Tagging Tasks": None,
"A Simple and Effective Model for Answering Multi-span Questions": None,
"Don't Use English Dev: On the Zero-Shot Cross-Lingual Evaluation of Contextual Embeddings": None,
"A* Beam Search": None,
"{M}u{VER}: {I}mproving First-Stage Entity Retrieval with Multi-View Entity Representations": None,
"{PRIDE}: {P}redicting {R}elationships in {C}onversations": None,
"Are {T}ransformers a Modern Version of {ELIZA}? {O}bservations on {F}rench Object Verb Agreement": None,
"{BiQUE}: {B}iquaternionic Embeddings of Knowledge Graphs": None,
"TADPOLE: {T}ask {AD}apted {P}re-Training via An{O}maLy D{E}tection": None,
"{{SHAPE}}: {S}hifted Absolute Position Embedding for Transformers": None,
"How much coffee was consumed during EMNLP 2019? Fermi Problems: A New Reasoning Challenge for AI": None,
"Reason{BERT}: {P}re-trained to Reason wit

In [66]:
 df.to_csv("../data/cl_papers.csv")

In [88]:
title_to_semantic_scholar_id = {
"Let's Stop Incorrect Comparisons in End-to-end Relation Extraction!": '49f203a3a22b662fb134c873f3293418ceaee7b3',
"DAGA: Data Augmentation with a Generation Approach forLow-resource Tagging Tasks": '6db4917fef590368dd523255ee9b3d9c8c4a3c53',
"A Simple and Effective Model for Answering Multi-span Questions": 'cc9afe82dd74486598023dcad37d4c8b810f15ac',
"Don't Use English Dev: On the Zero-Shot Cross-Lingual Evaluation of Contextual Embeddings": 'af0952325d1b7b100ae5280e7e97bd297765baab',
"A* Beam Search": 'a6e6197fcbb5bbe7d332904f2624e7bf1687c024',
"{M}u{VER}: {I}mproving First-Stage Entity Retrieval with Multi-View Entity Representations": 'f7ac2956642e58b8ed6290d0060763bbb5ddd21e',
"{PRIDE}: {P}redicting {R}elationships in {C}onversations": 'd248773e715f42e11d5df4366a5cdd3ae0cd8afe',
"Are {T}ransformers a Modern Version of {ELIZA}? {O}bservations on {F}rench Object Verb Agreement": '78d68e41d190f844f65e41ba79a8bae0714b8ab2',
"{BiQUE}: {B}iquaternionic Embeddings of Knowledge Graphs": '190f71c6f835f3040cbe2be7f15b05b117dced9d',
"TADPOLE: {T}ask {AD}apted {P}re-Training via An{O}maLy D{E}tection": '77994c9c7ec19a07173b88fb45a8900b6616dd03',
"{{SHAPE}}: {S}hifted Absolute Position Embedding for Transformers": '64522a5b3476e9f201f6a5b3e312ef0005c562f1',
"How much coffee was consumed during EMNLP 2019? Fermi Problems: A New Reasoning Challenge for AI": '4bbafce8de5301222f236784dfa23217636963e4',
"Reason{BERT}: {P}re-trained to Reason with Distant Supervision": '35ab270362e3dd4a1c95b63f7e800d5f7d4b9dc0',
"{I}ndo{NLI}: A Natural Language Inference Dataset for {I}ndonesian": '35772d76f90bdf74b9d49ec8e476fe27679022af',
"{MLEC-QA}: {A} {C}hinese {M}ulti-{C}hoice {B}iomedical {Q}uestion {A}nswering {D}ataset": '31acfba3a19f780a3239925ff12a7a4047d6a705',
"{W}ino{L}ogic: {A} Zero-Shot Logic-based Diagnostic Dataset for {W}inograd {S}chema {C}hallenge": 'b6588d413efc23122f690ace43a28db693d857c2',
"{N}ews{CLIP}pings: {A}utomatic {G}eneration of {O}ut-of-{C}ontext {M}ultimodal {M}edia": '89e7626d9b94e05c4154e7f6fb6df146fa4eb8fa',
"R^3Net:Relation-embedded Representation Reconstruction Network for Change Captioning": 'f8b9d8d187413d758d7849e96288838b5c1b75cf',
"Region under {D}iscussion for visual dialog": '4656b3fbc0e5dad904001de2eca8645813871801',
"Systematic Generalization on g{SCAN}: {W}hat is Nearly Solved and What is Next?": 'd41417f22f898125c4d34f672938ebb2a3764961',
"{ARMAN}: {P}re-training with {S}emantically {S}electing and {R}eordering of {S}entences for {P}ersian {A}bstractive {S}ummarization": '3e4c671a59939f2893787ad89b61245cc42081e5',
"Learning to Ask Conversational Questions by Optimizing Levenshtein Distance": 'cda69a6d4c3f087d3f8b7b0b40debdeea2bad727',
"Let’s Play mono-poly: BERT Can Reveal Words’ Degree of Polysemy": 'befbef9f4e4c6269fa712294430ff916cf2fd51c',
"MemPrompt: Memory-assisted Prompt Editing with User Feedback": 'f76c965153398cd8513ef95eaa32196c4cae3f86',
"Kernelized Concept Erasure": '4ca0f16524104777b001247662ef8d406d17ea3d',
"An Information-theoretic Approach to Prompt Engineering Without Ground Truth Labels": None,
"Causal Probing for Grammatical Number: From Encoding to Usage": '3a326123523f715f6eb74d12e38ccf7a4180dc46',
"Comprehensive Investigation of Multi-task Argument Mining": '4c41a1d8c79f57d2201bea3df115f607a518e433',
"Baize: An Open-Source Chat Model with Parameter-Efficient Tuning on Self-Chat Data": None,
"Vicarious Offense and Noise Audit of Offensive Speech Classifiers: Unifying Human and Machine Disagreement on What is Offensive": '546336b5a0781c88727750b6dcb4b7ff2ea7d7ae',
"Order-Theoretic Structured Prediction: Partially Ordering Tokens within a String": '340ae8b712340d513f288a3b84416f29bde1de2b',
"LLM-powered Data Augmentation for Enhanced Cross-lingual Performance": '91b2b47cabd800ef658b65bfe1f52b7293a740c3',
"CAPSTONE: Curriculum Sampling for Dense Retrieval with Document Expansion": '7a5c061f373d91c7865f227053ed3cb1cd99bbfa',
"MULTITuDE: Large-Scale Multilingual Machine-Generated Text Detection Benchmark": None,
"Enhancing Long-form Text Generation Efficacy with Task-adaptive Tokenization": '9a4f929b44345b9d547f8d2e1700caaab99237c0',
"Impressions: Visual Semiotics and Aesthetic Impact Understanding": 'f542e0e6df7ff8e2a669ca2190bd6559f0b376df',
"The Generalized Left-Corner Transformation": '8ad2d7c2f490658fd895a1cd05eb56e03924f21f',
"Hop, Union, Generate: Explainable Multi-hop Reasoning without Rationale Supervision": '891db732097183ee7e843ca134ed96d8f720f73c',
"PhenotypeCLIP: Phenotype-based Contrastive Learning for Medical Imaging Report Generation": '14d94e4a778496d3fb4a92e9effb7bd6f826f45e',
"Mapping Text to Knowledge Graph Entities by Vector Space Transformation": '33b60f5493e1a1cb303dd33569925e0ed0c7e189',
"Deep and Wide Reader: Effective Memory Augmenting Method for Large-Scale Reading Comprehension": 'b42e784ee14709828d9f1028f4147fb904d0be7c',
"Part-of-Speech Tagging for Twitter by Different Expression Styles": 'd386589b31d6948fcdb8bca7377bf0a288a898dc',
"Modeling Inter-Aspect Relation in Aspect-Based Sentiment Analysis": 'c77947a8b69d77269f7129ee937f00122b46e26e',
"Spider: A Large-Scale Complex Human-Labeled SQL Corpus for Semantic Parsing and Text-to-SQL Generation": '8e773b1840b894603c06b677a0f15ebcf0f26378',
"Process Paragraph Comprehension using World Knowledge": '190eb1dff4c08fc12a085242b67c0442cfe5fc84',
"TwoWingOS: Claim Entailment with Precise Evidence via a Two-Wing Optimization Strategy": '8cd595ee1d8b8ff5716f74b9223d736d21d12ce1',
"Commonsense Action Explanation in Human-Agent Communication": 'fa33e20a3265d9a506c11a392cde9c367c30284e',
"Text, Visual and Acoustic are Friends! A Multi-Modal Attention Framework for Utterance-Level Sentiment Prediction": 'ec99eca7c86601b8c909147f8caa8c35975bc44e',
"Task Oriented Parsing": '472a5227279b45f25508017816af34e3cb3ac0d7',
"Crosslingual Sense Embeddings in One Space": '21ea974bc645962f91d071984ea9c04ccf476c8e',
"Understanding Back-Translation at Scale": None,
"Adaptive Multi-pass Decoder for Neural Machine Translation with Reinforcement Learning": 'eea7aeb93f4137bee3e49deff7d3a142284efa2a',
"The Importance of Joint Multilingual Supervision for Cross-lingual Entity Linking": '41abb43d9d809e37946b91b8111c825778dc6e09',
"Cube Pruning for Neural Machine Translation": '95f5e548441ae7231a318f0ca739f9f53adfacd1',
"Natural Language Processing Not-At-All from Scratch: Evaluating The Utility of Hand-crafted Features in Deep Learning": '462d4e265c9cbe9ad5feeb9a7736184a90b36fed',
"Social IQa: Commonsense Reasoning about Social Interactions": None,
"When a Good Translation is Wrong in Context: Context-Aware Machine Translation Improves on Deixis, Ellipsis, and Lexical Cohesion": None,
"Modeling Visual Relationships in Language": '4eb2e2b9c22cb1da8561044ca0dc8fc0b13e3157',
"Retrieval-Enhanced Adversarial Training for Neural Response Generation": '5f7690abab9304949b62ee7ee44f0a1a52be90e7',
"RED<sup>FM</sup>: a Filtered and Multilingual Relation Extraction Dataset": '3c331785f7f5629563fbc4aabfe973d7c0dc57f7',
}

df['new_semantic_scholar_id'] = df['title'].map(title_to_semantic_scholar_id)

df['semantic_scholar_id'] = df.apply(
    lambda row: row['new_semantic_scholar_id'] if pd.notna(row['new_semantic_scholar_id']) else row['semantic_scholar_id'],
    axis=1
)
df.drop(columns=['new_semantic_scholar_id'], inplace=True)
